Check the status of tube lines and overground
https://api.tfl.gov.uk/line/mode/tube,overground,dlr,tflrail/status

29-6 added overground state

In [1]:
import requests
import requests_mock
import json
import pprint
import unittest
import sys

In [2]:
def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

In [3]:
# List of allowed lines
lines = ['Bakerloo',
 'Central',
 'Circle',
 'District',
 'DLR',
 'Hammersmith & City',
 'Jubilee',
 'London Overground',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'TfL Rail',
 'Victoria',
 'Waterloo & City']

In [4]:
ls

Tube requests 29-6-2017.ipynb  tube_state.json
test_tube_state.py


In [5]:
len(lines)

14

# Basic function of API

In [6]:
#url = 'https://api.tfl.gov.uk/line/mode/tube/status'   # tube only
url = 'https://api.tfl.gov.uk/line/mode/tube,overground,dlr,tflrail/status'
data = requests.get(url).json() # get the response object

# Cached data
with requests_mock.mock() as m:
    m.get(url, text=open('tube_state.json').read())
    #data = requests.get(url).json()
    
print_json(len(data))
print_json(data[0]['name'])
print_json(data[0]['lineStatuses'][0]['statusSeverityDescription'])

14
'Bakerloo'
'Good Service'


In [7]:
_lines = [line['name'] for line in data]
_lines

['Bakerloo',
 'Central',
 'Circle',
 'District',
 'DLR',
 'Hammersmith & City',
 'Jubilee',
 'London Overground',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'TfL Rail',
 'Victoria',
 'Waterloo & City']

In [8]:
data_dict = dict.fromkeys(_lines)    # Create the empty dict for the data
data_dict

{'Bakerloo': None,
 'Central': None,
 'Circle': None,
 'DLR': None,
 'District': None,
 'Hammersmith & City': None,
 'Jubilee': None,
 'London Overground': None,
 'Metropolitan': None,
 'Northern': None,
 'Piccadilly': None,
 'TfL Rail': None,
 'Victoria': None,
 'Waterloo & City': None}

Main logic for parsing the data into a useful dict

In [9]:
for line in data:                     # Assign data via key
    statuses = [status['statusSeverityDescription'] for status in line['lineStatuses']]
    state = ' + '.join(sorted(set(statuses)))
    
    if state == 'Good Service':   # if good status, this is the only status returned
        reason =  'Nothing to report'
    else:
        reason = ' *** '.join([status['reason'] for status in line['lineStatuses']])

    attr = {'State': state, 'Description': reason}
    data_dict[line['name']] = attr
    
print_json(data_dict)

{'Bakerloo': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Central': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Circle': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'DLR': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'District': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Hammersmith & City': {'Description': 'Nothing to report',
                        'State': 'Good Service'},
 'Jubilee': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'London Overground': {'Description': 'London Overground: Minor delays '
                                      'Richmond to Stratford and Willesden '
                                      'Junction to Clapham Junction while we '
                                      'fix a faulty train at Richmond, GOOD '
                                      'SERVICE all other routes. ',
                       'State': 'Minor Delays'},
 'Metropolitan': {'

## Create function for parsing response data

In [10]:
def parse_api_response(response):
    """Take in the TFL API json."""
    lines = [line['name'] for line in response]    # All available lines
    data_dict = dict.fromkeys(lines)
    
    for line in response:                     # Assign data via key
        statuses = [status['statusSeverityDescription'] for status in line['lineStatuses']]
        state = ' + '.join(sorted(set(statuses)))
    
        if state == 'Good Service':   # if good status, this is the only status returned
            reason =  'Nothing to report'
        else:
            reason = ' *** '.join([status['reason'] for status in line['lineStatuses']])

        attr = {'State': state, 'Description': reason}
        data_dict[line['name']] = attr
    
    return data_dict

In [11]:
## Mock data
with requests_mock.mock() as m:
    url = 'https://api.tfl.gov.uk/line/mode/tube,overground,dlr,tflrail/status'
    m.get(url, text=open('tube_state.json').read())
    response = requests.get(url).json()

class TestLondonTubeSensor(unittest.TestCase):

    def test_parse_API_response(self):
        data = parse_api_response(response)
        
        self.assertEqual(data['London Overground']['State'],
                        'Minor Delays')
        
        
unittest.main(argv=['ignored', '-v'], exit=False)

test_parse_API_response (__main__.TestLondonTubeSensor) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# Create tube class

Wrap in a class with a method for updating and another for returning line of interest. Follow
https://github.com/robmarkcole/home-assistant/blob/dev/homeassistant/components/sensor/bitcoin.py

In [12]:
class TubeData(object):
    """Get the latest tube data from TFL."""
    url = 'https://api.tfl.gov.uk/line/mode/tube/status'
    
    def __init__(self):
        """Initialize the data object."""
        self.data = None

    def update(self):
        """Get the latest data from TFL."""
        response = requests.get(url)
        
        if response.status_code != 200:
            #_LOGGER.warning("Invalid response from API")
            print("Invalid response from API")
        else:
            self.data = parse_api_response(response.json())

In [13]:
test_tube = TubeData()
test_tube.update()
test_tube.data

{'Bakerloo': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Central': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Circle': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'DLR': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'District': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Hammersmith & City': {'Description': 'Nothing to report',
  'State': 'Good Service'},
 'Jubilee': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'London Overground': {'Description': 'London Overground: Minor delays Richmond to Stratford and Willesden Junction to Clapham Junction while we fix a faulty train at Richmond, GOOD SERVICE all other routes. ',
  'State': 'Minor Delays'},
 'Metropolitan': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Northern': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Piccadilly': {'Description': 'Nothing to report', 'State': 'Good Ser

In [14]:
class LondonTubeSensor():    # Entity
    """
    Sensor that reads the status of a tube line.
    """

    ICON = 'mdi:subway'

    def __init__(self, name, data):
        """Initialize the sensor."""
        self._name = name             # the name of the line from the allowed list
        self._data = data
        self._state = None
        self._description = None

    @property
    def name(self):
        """Return the name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    @property
    def device_state_attributes(self):
        """Return other details about the sensor state."""
        attrs = {}  # {'attribution': 'Data provided by transportapi.com'}
        attrs['Description'] = self._description # if there is data, append
        return attrs

    def update(self):
        """Update the sensor."""
        self._data.update()      # update the data object
        self._state = self._data.data[self.name]['State']
        self._description = self._data.data[self.name]['Description']

In [15]:
data = TubeData()   # init a data object
bakerloo_sensor = LondonTubeSensor('Bakerloo', data)
bakerloo_sensor.update()
print(bakerloo_sensor.state)
print(bakerloo_sensor.device_state_attributes['Description'])

Good Service
Nothing to report


## Example usage

In [16]:
CONF_LINE = 'line'
config = {CONF_LINE:['Bakerloo', 'District']}

def example_setup_platform(config):
    """Set up the Bitcoin sensors."""


    data = TubeData()
    sensors = []
    for line in config[CONF_LINE]:
        sensors.append(LondonTubeSensor(line, data))

    return sensors

In [17]:
sensors = example_setup_platform(config)

for sensor in sensors:
    sensor.update()
    print(sensor.name)
    print(sensor.state)
    print('*****')

Bakerloo
Good Service
*****
District
Good Service
*****


In [18]:
sensors[0]._data.data

{'Bakerloo': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Central': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Circle': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'DLR': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'District': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Hammersmith & City': {'Description': 'Nothing to report',
  'State': 'Good Service'},
 'Jubilee': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'London Overground': {'Description': 'London Overground: Minor delays Richmond to Stratford and Willesden Junction to Clapham Junction while we fix a faulty train at Richmond, GOOD SERVICE all other routes. ',
  'State': 'Minor Delays'},
 'Metropolitan': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Northern': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Piccadilly': {'Description': 'Nothing to report', 'State': 'Good Ser

In [19]:
sensors[0]._data.data = 'test'
sensors[0]._data.data

'test'

In [20]:
sensors[1]._data.data

'test'

## Save response to json for testing

In [21]:
url = 'https://api.tfl.gov.uk/line/mode/tube,overground,dlr,tflrail/status'
response = requests.get(url)  # get the response object
filename = 'tube_state.json'

with open(filename, 'wb') as fd:                 # write the response to file. Can format the .json file but no blank lines allowed 
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)

data = response.json()[0]['lineStatuses']       # data is a list of statuses on the line
#print_json(data)